In [1]:
# !CUDA_VISIBLE_DEVICES=1
import pickle
import time
from scipy import stats
import numpy as np

import torch
torch.cuda.set_device(0) 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split

from transformers import BertConfig, BertModel, BertTokenizer, BertPreTrainedModel, AdamW, \
                         get_linear_schedule_with_warmup, WEIGHTS_NAME, CONFIG_NAME

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_len = 256

In [ ]:
save_path = './phrase_model_data.pkl'
with open(save_path, 'rb') as f:
    traindata = pickle.load(f)
len(traindata)

In [32]:
def prepare_data(tempdata, phrase_rule_133):
    lnis = []
    labels = []
    inputs = []
    num_classes = len(phrase_rule_133)
    for lni, items in tempdata.items():
        label = [0] * num_classes
        phrase = [phrase_rule_133[p] for p in items['phrases'] if p in phrase_rule_133]
        if not phrase:
            continue

        if len(items['pred_sents']) > 6:
            sent = ' '.join(s for _, s in items['pred_sents'][:3] + items['pred_sents'][-3:])
        else:
            sent = ' '.join(s for _, s in items['pred_sents'])

        if not sent:
            continue

        inputs.append(sent)

        for i in set(phrase):
            label[i] = 1
        assert sum(label) > 0
        labels.append(label[:])
        lnis.append(lni)

    print(len(labels), len(inputs), len(lnis))
    return lnis, inputs, labels

# _, inputs, labels = prepare_data(traindata, phrase_rule_133)
# inputs[10]

In [33]:
# test_path = './phrase_model_data_ftc.pkl'
# with open(test_path, 'rb') as f:
#     testdata = pickle.load(f)
# print(len(testdata))

test_lnis, test_inputs, test_labels = prepare_data(testdata, phrase_rule_133)
test_inputs[10]

93581 93581 93581


"extension of time granted to april , to file the reply to the informal response to the petition for writ of habeas corpus . extension is granted based upon deputy state public defender debra s . sabah press's representation that she anticipates filing that brief by ."

In [18]:
def bert_tokenize_ids(inputs, tokenizer):
    input_ids = [] # List[List[int]]

    # For every sentence...
    for i, sent in enumerate(inputs):
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                            # This function also supports truncation and conversion
                            # to pytorch tensors, but we need to do padding, so we
                            # can't use these features :( .
                            # max_length = 128,          # Truncate all sentences.
                            #return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_sent)
    # print(len(input_ids))
    return input_ids

test_input_ids = bert_tokenize_ids(test_inputs, tokenizer)

In [19]:
len(test_input_ids)

93581

In [9]:
print(input_ids[10])
lengths = [len(x) for x in input_ids]
stats.describe(lengths)

[101, 15339, 4267, 2038, 6406, 1037, 4367, 2005, 2019, 5331, 1997, 2051, 2000, 5371, 17304, 2000, 1996, 14351, 3648, 1005, 1055, 3818, 9556, 1998, 6749, 22137, 1012, 2008, 4367, 2003, 4379, 1012, 6254, 1012, 102]


DescribeResult(nobs=320635, minmax=(3, 10479), mean=126.5668626319647, variance=12403.922152096984, skewness=3.800628086074086, kurtosis=238.0777072156434)

In [23]:
def prepare_attention_mask(new_inp_ids, pad_id):
    assert pad_id == 0, 'bug of pad id'
    # Create attention masks
    attention_masks = []

    # For each sentence...
    for sent in new_inp_ids:
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > pad_id) for token_id in sent]

        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)
    print('Length of Attention mask : {}'.format(len(attention_masks)))
    return attention_masks


def truncate_pad_id(input_ids, tokenizer, max_len=256):
    half_len = max_len // 2

    # truncation and padding
    pad_id = tokenizer.pad_token_id # 0

    new_inp_ids = []
    for x in input_ids:
        ll = len(x)
        if ll > max_len:
            new_inp_ids.append(x[:half_len] + x[-half_len:])
        elif ll < max_len:
            new_inp_ids.append(x + [pad_id] * (max_len-ll))
        else:
            new_inp_ids.append(x)
        assert len(new_inp_ids[-1]) == max_len

    print('Length of inputs : {}'.format(len(new_inp_ids)))
    attention_masks = prepare_attention_mask(new_inp_ids, pad_id)
    return new_inp_ids, attention_masks

new_test_inp_ids, test_attention_masks = truncate_pad_id(test_input_ids, tokenizer)

Length of inputs : 93581
Length of Attention mask : 93581


In [41]:
# test_inputs = torch.tensor(new_test_inp_ids).long()
# test_masks = torch.tensor(test_attention_masks).long()
# test_labels = torch.tensor(test_labels).float()

# test_data = TensorDataset(test_inputs, test_masks, test_labels)
# test_sampler = SequentialSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=8)
# len(test_dataloader)

11698

In [42]:
for b in test_dataloader:
    fi = b[0]
    fl = b[2]
    break


In [15]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(new_inp_ids, labels, 
                                                            random_state=2020, test_size=0.15)
# Do the same for the masks.
# train_masks, val_masks, _, _ = train_test_split(attention_masks, y,
#                                              random_state=2020, test_size=0.15)

In [19]:
len(train_labels), len(val_labels)

(272539, 48096)

In [21]:
# Do the same for the masks.
train_masks, val_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2020, test_size=0.15)
len(val_masks)

48096

In [44]:
# bert_process_data = {'train_inputs': train_inputs, 'val_inputs': val_inputs, 'test_inputs': test_inputs, 
#                      'train_labels': train_labels, 'val_labels': val_labels, 'test_labels': test_labels, 
#                      'train_masks': train_masks, 'val_masks': val_masks, 'test_masks': test_masks, 
#                      'test_lnis': test_lnis}
# with open('./e3_bert_process_data.pkl', 'wb') as f:
#     pickle.dump(bert_process_data, f)

In [2]:
# loading data
with open('./e3_bert_process_data.pkl', 'rb') as f:
    bert_process_data = pickle.load(f)


# train_inputs = bert_process_data['train_inputs']
# train_labels = bert_process_data['train_labels']
# train_masks = bert_process_data['train_masks']

val_inputs = bert_process_data['val_inputs']
val_labels = bert_process_data['val_labels']
val_masks = bert_process_data['val_masks']

test_inputs, test_masks = bert_process_data['test_inputs'], bert_process_data['test_masks']
test_labels, test_lnis = bert_process_data['test_labels'], bert_process_data['test_lnis']

In [3]:
train_inputs.size(), val_inputs.size(), test_inputs.size(), train_masks.size()

NameError: name 'train_inputs' is not defined

In [4]:
batch_size = 16

# train_inputs = torch.tensor(train_inputs).long()
# val_inputs = torch.tensor(val_inputs).long()

# train_labels = torch.tensor(train_labels).float()
# val_labels = torch.tensor(val_labels).float()

# train_masks = torch.tensor(train_masks).long()
# val_masks = torch.tensor(val_masks).long()

# test_inputs = torch.tensor(new_test_inp_ids).long()
# test_masks = torch.tensor(test_attention_masks).long()
# test_labels = torch.tensor(test_labels).float()



# Create the DataLoader for our training set.
# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(val_inputs, val_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
len(test_dataloader)

5849

In [5]:
# class FtBert(nn.Module):
#     def __init__(self, hidden_size=768, num_labels=133, dropout_prob=0.5):
#         super(FtBert, self).__init__()
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased', 
#             cache_dir='/home/aidog/workspace/charley/sound/toxic_comment_c/bert')
#         self.dropout = nn.Dropout(dropout_prob)
#         self.classifier = nn.Linear(hidden_size, num_labels)
        
#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert_model(input_ids, attention_mask=attention_mask)
#         pooled_output = outputs[1]
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
#         return logits
    

class FtBert(nn.Module):
    def __init__(self, hidden_size=768, num_labels=133, dropout_prob=0.5):
        super(FtBert, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(hidden_size, num_labels)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [6]:
model = FtBert()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
PATH = './ft_bert_0224.pt'
model.load_state_dict(torch.load(PATH))
print('finished loading')

finished loading


In [6]:
class FtBert(BertPreTrainedModel):
    def __init__(self, config, hidden_size=768, num_labels=133, dropout_prob=0.5):
        super(FtBert, self).__init__(config)
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Linear(hidden_size, num_labels)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [7]:
# configure = BertConfig()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # device = torch.device('cpu')
# model = FtBert(configure)
# model.to(device)

In [10]:
criterion = nn.BCEWithLogitsLoss()

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [13]:
def train_epoch(model, device, epoch, train_dataloader, validation_dataloader, 
                criterion, optimizer, clip=5.):
    model.train()
    train_loss = 0
    t0 = time.time()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader, 1):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        loss = criterion(outputs, b_labels)
        train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        # Update the learning rate.
        # scheduler.step()
        if step % 1000 == 0:
            # print loss info every 20 Iterations
            log_str = "Epoch : {} , Iteration : {} , Time : {:.2f} , TrainLoss : {:.4f}".format \
                        (epoch, step, time.time()-t0, train_loss/step)
            print(log_str)
            t0 = time.time()
        train_loss /= len(train_dataloader)

    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for step, batch in enumerate(validation_dataloader, 1):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids, attention_mask=b_input_mask)
            loss = criterion(outputs, b_labels)
            eval_loss += loss.item()
        eval_loss /= len(validation_dataloader)

    return model, optimizer, train_loss, eval_loss

In [14]:
n_epochs = 3
clip = 2.0
save_path = './ft_bert_0217.pt'
best_eval_loss = float('inf')
for epoch in range(1, 1+n_epochs):
    model, optimizer, train_loss, eval_loss = train_epoch(model, device, epoch, 
                                                          train_dataloader, validation_dataloader, 
                                                          criterion, optimizer, clip=clip)

    print(">> Epoch : {} , TrainLoss : {:.4f} , EvalLoss : {:.4f}\n".format \
          (epoch, train_loss, eval_loss))

    if eval_loss < best_eval_loss:
        best_eval_loss = eval_loss
        torch.save(model.state_dict(), save_path)

Epoch : 1 , Iteration : 1000 , Time : 324.37 , TrainLoss : 0.0001
Epoch : 1 , Iteration : 2000 , Time : 324.54 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 3000 , Time : 326.64 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 4000 , Time : 327.24 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 5000 , Time : 326.42 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 6000 , Time : 326.00 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 7000 , Time : 326.05 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 8000 , Time : 326.23 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 9000 , Time : 326.26 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 10000 , Time : 326.65 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 11000 , Time : 326.82 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 12000 , Time : 326.87 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 13000 , Time : 326.97 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 14000 , Time : 325.77 , TrainLoss : 0.0000
Epoch : 1 , Iteration : 15000 , Time : 327.17 , TrainLoss : 0.0000
Epoc

In [7]:
def predict_res(model, data_loader, device):
    model.eval()
    y_true = None
    y_pred = None
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids, attention_mask=b_input_mask)
            
            if y_true is None:
                y_true = b_labels
            else:
                y_true = torch.cat((y_true, b_labels), 0)
            # outputs = model(seqs, seq_lens)

            if y_pred is None:
                y_pred = outputs
            else:
                y_pred = torch.cat((y_pred, outputs), 0)
            # break

    print(y_true.size(), y_pred.size())
    y_pred = torch.sigmoid(y_pred)
    return y_true.cpu().numpy(), y_pred.cpu().numpy()


In [8]:
def mean_column_wise_auc(y_true, y_pred):
    assert y_true.shape[1] == y_pred.shape[1],'Arrays must have the same dimension'
    list_of_aucs = []
    for column in range(y_true.shape[1]):
        #print(sum(y_true[:,column]), sum(y_pred[:,column]))
        if sum(y_true[:,column]) == 0:
            continue
        list_of_aucs.append(roc_auc_score(y_true[:,column],y_pred[:,column]))
    # print(list_of_aucs)
    return np.array(list_of_aucs).mean(), len((list_of_aucs))

def fit_active_value(s1, active_value):
    # return np.round(s1)
    return (s1 > active_value).astype(int)

def cut_max_num(s1, max_num):
    max_ids = np.argsort(s1)[-max_num:]
    s3 = np.zeros_like(s1)
    # for i in max_ids:
    #     s3[i] = 1
    s3[max_ids] = 1
    return s3

def cal_avg_p_r(arr_true, arr_pred, max_num=6, active_value=0.05):
    ps, rs = [], []
    for i in range(arr_true.shape[0]):
        t1, s1 = arr_true[i], arr_pred[i]
        if sum(t1) <= 0:
            continue
        s2 = fit_active_value(s1, active_value)
        if sum(s2) > max_num:
            s2 = cut_max_num(s1, max_num)
        # if sum(s2) > 4:
        #     s2 = get_maxf_adj(s1, s2, adj_phrase_map, phrase_prob, cannot_be_only)
        # s2 = fit_threshold(s1, thresholds)

        p, r = precision_score(t1, s2), recall_score(t1, s2)
        ps.append(p)
        rs.append(r)
    return np.average(ps), np.average(rs), len(ps), ps, rs

In [9]:
val_true, val_pred = predict_res(model, validation_dataloader, device)

torch.Size([48096, 133]) torch.Size([48096, 133])


In [10]:
# test_dataloader
test_true, test_pred = predict_res(model, test_dataloader, device)

torch.Size([93581, 133]) torch.Size([93581, 133])


In [11]:
val_auc = roc_auc_score(val_true, val_pred)
test_auc = mean_column_wise_auc(test_true, test_pred) # 0.9775929439549857, 129
# val_auc # 0.9718519698062188 -> 0.9726565791687066 # val
# test_auc = roc_auc_score(test_true, test_pred)
val_auc, test_auc

(0.9712245081369616, (0.9785304744549129, 129))

In [12]:
avg_p, avr_r, num, ps, rs = cal_avg_p_r(val_true, val_pred, max_num=6, active_value=0.02)
avg_p, avr_r, num

(0.5286673181414948, 0.9365043557065764, 48096)

In [14]:
tavg_p, tavr_r, tnum, tps, trs = cal_avg_p_r(test_true, test_pred, max_num=6, active_value=0.02)
tavg_p, tavr_r, tnum # (0.5230689118071864, 0.9624222365379759, 93581)

(0.5222283369487396, 0.958959066612864, 93581)

In [13]:
np.mean(np.concatenate((ps, tps))), np.mean(np.concatenate((rs, trs)))

NameError: name 'tps' is not defined

In [22]:
# 0.05  (0.6115475516068873, 0.9200020758747057, 48096)
# 0.03  (0.5474526120430204, 0.9372397730993041, 48096)
# 0.025 (0.5391036126577576, 0.9401866241062848, 48096)
# 0.02  (0.508811839430167, 0.9462131622997392, 48096)
len(validation_dataloader)

3006

# sampling

In [10]:
def rand_sampling(data, num=50):
    _arr = np.asarray(data)
    ids = np.random.choice(len(_arr), num, replace=False)
    return _arr[ids].tolist()

sample_ids = rand_sampling(range(93580), 1000)

In [11]:
test_inputs_ = test_inputs.tolist()
test_masks_ = test_masks.tolist()
test_labels_ = test_labels.tolist()

sample_lnis = []
sample_inputs_ = []
sample_masks_ = []
sample_labels_ = []

for i in sample_ids:
    sample_labels_.append(test_labels_[i])
    sample_inputs_.append(test_inputs_[i])
    sample_masks_.append(test_masks_[i])
    sample_lnis.append(test_lnis[i])
    
print(len(sample_labels_))

1000


In [12]:
sample_inputs = torch.tensor(sample_inputs_).long()
sample_masks = torch.tensor(sample_masks_).long()
sample_labels = torch.tensor(sample_labels_).float()

sample_data = TensorDataset(sample_inputs, sample_masks, sample_labels)
sample_sampler = SequentialSampler(sample_data)
sample_dataloader = DataLoader(sample_data, sampler=sample_sampler, batch_size=10)

In [13]:
import time 
# cpu : 1000 - 214s
start_time = time.time()
sample_true, sample_pred = predict_res(model, sample_dataloader, device)
print(time.time() - start_time)

torch.Size([1000, 133]) torch.Size([1000, 133])
213.7441337108612


In [24]:
mean_column_wise_auc(sample_true, sample_pred) # 0.9775929439549857, 129

(0.9808125936962412, 108)

In [25]:
avg_p, avr_r, num, ps, rs = cal_avg_p_r(sample_true, sample_pred, max_num=6, active_value=0.02)
avg_p, avr_r, num,

(0.54462, 0.9613180952380952, 5000)

In [10]:
def prepare_samle_res(sample_lnis, sample_true, sample_pred, i2phrase, max_num=6, active_value=0.05):
    sample_show_res = []
    for i, lni in enumerate(sample_lnis):
        t1, s1 = sample_true[i], sample_pred[i]
        if sum(t1) <= 0:
            continue
        s2 = fit_active_value(s1, active_value)
        if sum(s2) > max_num:
            s2 = cut_max_num(s1, max_num)

        tids = np.where(t1 == 1)[0]
        pids = np.where(s2 == 1)[0]

        p, r = precision_score(t1, s2), recall_score(t1, s2)

        true_p = ' # '.join(i2phrase[j] for j in tids)
        pred_p = ' # '.join(i2phrase[j] for j in pids)
        sample_show_res.append((lni, true_p, pred_p, p, r, len(tids), len(pids)))

    return sample_show_res

# sample_show_res = prepare_samle_res(sample_lnis, sample_true, sample_pred, i2phrase_rule_133, max_num=6, active_value=0.02)
# print(len(sample_show_res))
# print(sample_show_res[100])

## is relate back

In [36]:
# cal_acc_relate_back(arr_true, arr_pred)
tp, fp, tn, fn = 0, 0, 0, 0
active_value = 0.02
ps, rs = [], []
for i in range(val_true.shape[0]):
    t1, s1 = val_true[i], val_pred[i]
    if sum(t1) <= 0:
        continue
    s2 = fit_active_value(s1, active_value)
    tbool = is_relate_back(t1, i2phrase_rule_133, phrase_rule)
    sbool = is_relate_back(s2, i2phrase_rule_133, phrase_rule)
    if tbool and sbool:
        tp += 1
    elif tbool:
        fn += 1
    elif sbool:
        fp += 1
    else:
        tn += 1

tp, fp, tn, fn, tp+fp+tn+fn 

(19893, 7834, 19996, 373, 48096)

In [32]:
def is_relate_back(arr, i2phrase_rule_133, phrase_rule):
    ids = np.where(arr == 1)[0]
    bools = []
    for i in ids:
        p = i2phrase_rule_133[i]
        pinfos = phrase_rule[p]
        ibool = any([item.get('is_realte_back') == '1' for item in pinfos['letter'].values()])
        bools.append(ibool)
    return any(bools)

def showp(arr, i2phrase_rule_133):
    ids = np.where(arr == 1)[0]
    for i in ids:
        p = i2phrase_rule_133[i]
        print(p)

False

In [11]:
output_dir = "./bert_models/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FtBert.from_pretrained(output_dir)
model.to(device)

FtBert(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [12]:
val_true, val_pred = predict_res(model, validation_dataloader, device)

torch.Size([48096, 133]) torch.Size([48096, 133])


In [13]:
# test_dataloader
test_true, test_pred = predict_res(model, test_dataloader, device)

torch.Size([93581, 133]) torch.Size([93581, 133])


In [14]:
val_auc = roc_auc_score(val_true, val_pred)
test_auc = mean_column_wise_auc(test_true, test_pred)
val_auc, test_auc

(0.5018720268156032, (0.51028013505342, 129))

In [15]:
val_pr = cal_avg_p_r(val_true, val_pred, max_num=6, active_value=0.02)
val_pr[:3]

(0.016099744954535374, 0.07774541855442554, 48096)

In [ ]:
test_pr = cal_avg_p_r(test_true, test_pred, max_num=6, active_value=0.02)
test_pr[:3]